In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import requests

def fetch_article_list(params: dict) -> list[dict]:
    url = "https://m.land.naver.com/cluster/ajax/articleList"
    s = requests.Session()
    s.headers.update({
        "User-Agent": "Mozilla/5.0",
        "Referer": "https://m.land.naver.com/",
        "Accept": "application/json, text/plain, */*",
    })
    r = s.get(url, params=params, timeout=15)
    r.raise_for_status()
    data = r.json()

    # ── 안전 분기 ──
    body = data.get("body")  # ← 지금 케이스에서 존재
    if body is None:
        # 드물게 전체가 리스트/혹은 다른 래핑으로 올 수 있음
        if isinstance(data, list):
            items = data
        elif isinstance(data, dict):
            # articles가 최상위로 오는 변형 케이스
            items = data.get("articles", []) or data.get("list", [])
        else:
            items = []
    else:
        # body가 리스트 vs 딕셔너리 모두 처리
        if isinstance(body, list):
            items = body
        elif isinstance(body, dict):
            items = body.get("articles") or body.get("list") or []
        else:
            items = []

    return items, data.get("more"), data.get("page")

# 예시 파라미터
params = {
    "rletTpCd": "APT",            # APT/OPST/GM …
    "tradTpCd": "A1:B1:B2",       # A1=매매, B1=전세, B2=월세
    "z": "17",
    "lat": "37.4979",
    "lon": "127.0276",
    # 필요시: "cortarNo": "1168000000",
    "page": "1",
}

items, more, page = fetch_article_list(params)

# 필드가 자주 바뀌므로 키 존재 체크
for it in items:
    atcl_no = it.get("atclNo") or it.get("articleNo")
    name    = it.get("atclNm") or it.get("articleName")
    price   = it.get("prc")    or it.get("priceString")
    floor   = it.get("flrInfo") or it.get("floorInfo")
    realtor = it.get("rltrNm") or it.get("realtorName")
    print(atcl_no, name, price, floor, realtor)


In [3]:
import requests, math, time, random

S = requests.Session()
S.headers.update({
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://m.land.naver.com/",
    "Accept": "application/json, text/plain, */*",
})

params = {
    "view": "atcl",
    "rletTpCd": "APT",              # 아파트
    "tradTpCd": "A1:B1:B2",         # 매매/전세/월세
    "z": "17",
    "lat": "37.4979",
    "lon": "127.0276",
    "cortarNo": "1168000000",       # (선택) 강남구 예시
    # 지도 bbox를 넣으면 더 정확함:
    # "btm": "...", "lft": "...", "top": "...", "rgt": "..."
}

CLUSTER_URL = "https://m.land.naver.com/cluster/clusterList"
r = S.get(CLUSTER_URL, params=params, timeout=15)
r.raise_for_status()
cl = r.json()

# 클러스터에서 그룹(버블) 꺼내기
groups = (cl.get("data") or {}).get("ARTICLE") or []
print("그룹 개수:", len(groups))

그룹 개수: 500


In [4]:
groups

[{'lgeo': '2122110120210',
  'count': 167,
  'z': 17,
  'lat': 37.51992188,
  'lon': 127.05933838,
  'psr': 0.8,
  'tourExist': True},
 {'lgeo': '2122110122000',
  'count': 121,
  'z': 17,
  'lat': 37.52402344,
  'lon': 127.05705566,
  'psr': 0.8,
  'tourExist': False},
 {'lgeo': '2122110033121',
  'count': 90,
  'z': 17,
  'lat': 37.52587954,
  'lon': 127.05405782,
  'psr': 0.7,
  'tourExist': False},
 {'lgeo': '2120332322021',
  'count': 86,
  'z': 17,
  'lat': 37.49463742,
  'lon': 127.05818975,
  'psr': 0.7,
  'tourExist': False},
 {'lgeo': '2122110033113',
  'count': 85,
  'z': 17,
  'lat': 37.52475586,
  'lon': 127.05623627,
  'psr': 0.7,
  'tourExist': False},
 {'lgeo': '2120332302311',
  'count': 84,
  'z': 17,
  'lat': 37.48083364,
  'lon': 127.06434713,
  'psr': 0.7,
  'tourExist': False},
 {'lgeo': '2120332213310',
  'count': 83,
  'z': 17,
  'lat': 37.48091925,
  'lon': 127.05500343,
  'psr': 0.7,
  'tourExist': False},
 {'lgeo': '2120332213120',
  'count': 78,
  'z': 17,
 

In [5]:
ART_URL = "https://m.land.naver.com/cluster/ajax/articleList"
all_items = []

for g in groups:
    lgeo  = g["lgeo"]     # ← 그룹 식별자 (중요)
    count = int(g["count"])
    z2    = g["z"]
    lat2  = g["lat"]
    lon2  = g["lon"]

    pages = max(1, math.ceil(count / 20))   # 네이버가 20개 단위 반환

    for page in range(1, pages + 1):
        q = {
            "itemId": lgeo,         # = lgeo
            "mapKey": "",
            "lgeo": lgeo,
            "showR0": "",
            "rletTpCd": params["rletTpCd"],
            "tradTpCd": params["tradTpCd"],
            "z": z2,
            "lat": lat2,
            "lon": lon2,
            "totCnt": count,        # 그룹 총개수(중요)
            "cortarNo": params.get("cortarNo", ""),
            "page": str(page),
        }
        rr = S.get(ART_URL, params=q, timeout=15)
        rr.raise_for_status()
        try:
            data = rr.json()

            # 응답 래핑 표준화
            body = data.get("body")
            if isinstance(body, list):
                items = body
            elif isinstance(body, dict):
                items = body.get("articles") or body.get("list") or []
            else:
                items = data if isinstance(data, list) else data.get("articles", [])

            all_items.extend(items)
            time.sleep(random.uniform(0.8, 1.6))
        except : 
            continue

print("수집 매물 수:", len(all_items))
# 필드 예시 출력
for it in all_items[:5]:
    print(
        it.get("atclNo") or it.get("articleNo"),
        it.get("atclNm") or it.get("articleName"),
        it.get("prc") or it.get("priceString"),
        it.get("flrInfo") or it.get("floorInfo"),
        it.get("rltrNm") or it.get("realtorName"),
    )

수집 매물 수: 693
2555322289 아크로삼성 150000 22/25 오성공인중개사
2555148713 아크로삼성 800000 14/25 오성공인중개사
2554932198 아크로삼성 600000 5/25 타임부동산공인중개사사무소
2554938130 아크로삼성 210000 13/25 타임부동산공인중개사사무소
2554576675 아크로삼성 300000 11/25 (정문앞)청담디엘로공인중개사사무소


In [1]:
len(all_items)

NameError: name 'all_items' is not defined

In [1]:
for it in all_items[:100]:
    print(
        it.get("atclNo") or it.get("articleNo"),
        it.get("atclNm") or it.get("articleName"),
        it.get("prc") or it.get("priceString"),
        it.get("flrInfo") or it.get("floorInfo"),
        it.get("rltrNm") or it.get("realtorName"),
    )

NameError: name 'all_items' is not defined